# Airport Weather Alerts with Gemini

This notebook is the scheduled version of the Challenge 5 notebook, meant to run periodically. For this assignment it is set to run every 3 hours.



In [ ]:
# Import the file into a BigQuery table

from google.cloud import bigquery

client = bigquery.Client()

# Set dataset reference vars.
project_id = "qwiklabs-gcp-02-949c0486d822"
dataset_id = "dani_data_to_ai_workshop"
table_id = "airports"

dataset_ref = bigquery.DatasetReference(project_id, dataset_id)
dataset = bigquery.Dataset(dataset_ref)
client.create_dataset(dataset, exists_ok=True)

# Define the schema
schema = [
    bigquery.SchemaField("id", "INTEGER"),
    bigquery.SchemaField("ident", "STRING"),
    bigquery.SchemaField("type", "STRING"),
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("latitude_deg", "FLOAT"),
    bigquery.SchemaField("longitude_deg", "FLOAT"),
    bigquery.SchemaField("elevation_ft", "FLOAT"),
    bigquery.SchemaField("continent", "STRING"),
    bigquery.SchemaField("iso_country", "STRING"),
    bigquery.SchemaField("iso_region", "STRING"),
    bigquery.SchemaField("municipality", "STRING"),
    bigquery.SchemaField("scheduled_service", "STRING"),
    bigquery.SchemaField("icao_code", "STRING"),
    bigquery.SchemaField("iata_code", "STRING"),
    bigquery.SchemaField("gps_code", "STRING"),
    bigquery.SchemaField("local_code", "STRING"),
    bigquery.SchemaField("home_link", "STRING"),
    bigquery.SchemaField("wikipedia_link", "STRING"),
    bigquery.SchemaField("keywords", "STRING"),
]

# Create the table if it doesn't exist.
table_ref = bigquery.TableReference(dataset_ref, table_id)
table = bigquery.Table(table_ref, schema=schema)
client.create_table(table, exists_ok=True)

# Load the data into the table.
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip the header row
)

job = client.load_table_from_uri(
    'gs://labs.roitraining.com/data-to-ai-workshop/airports.csv',
    table,
    location='US',
    job_config=job_config
)

try:
    job.result()  # Wait for the job to complete
    table = client.get_table(table_ref)
    print(f"Loaded {table.num_rows} rows and {len(table.schema)} columns to {table_id}")
except Exception as e:
    print(f"Error loading data: {e}")


# Validate the table was created.
table = client.get_table(table_ref)
print(f"Table {table_id} created with {table.num_rows} rows and {len(table.schema)} columns")
